In [1]:
import os
import argparse
import re

import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def sorted_alphanum(file_list_ordered):
    """
    Sorts the list alphanumerically
    Args:
        file_list_ordered (list): list of files to be sorted
    Return:
        sorted_list (list): input list sorted alphanumerically
    """
    def convert(text):
        return int(text) if text.isdigit() else text

    def alphanum_key(key):
        return [convert(c) for c in re.split('([0-9]+)', key)]

    sorted_list = sorted(file_list_ordered, key=alphanum_key)

    return sorted_list
    
def get_file_list(path, extension=None):
    """
    Build a list of all the files in the provided path
    Args:
        path (str): path to the directory 
        extension (str): only return files with this extension
    Return:
        file_list (list): list of all the files (with the provided extension) sorted alphanumerically
    """
    if extension is None:
        file_list = [os.path.join(path, f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    else:
        file_list = [
            os.path.join(path, f)
            for f in os.listdir(path)
            if os.path.isfile(os.path.join(path, f)) and os.path.splitext(f)[1] == extension
        ]
    file_list = sorted_alphanum(file_list)

    return file_list

In [3]:
def scene_flow_EPE_np(pred, labels):
    error = np.sqrt(np.sum((pred - labels)**2, 1) + 1e-20)

    gtflow_len = np.sqrt(np.sum(labels*labels, 1) + 1e-20) # B,N
    acc1 = np.sum(np.logical_or((error <= 0.05), (error/gtflow_len <= 0.05)))
    acc2 = np.sum(np.logical_or((error <= 0.1), (error/gtflow_len <= 0.1)))
    acc3 = np.sum(np.logical_or((error > 0.3), (error/gtflow_len > 0.1)))

    num = pred.shape[0]
    acc1 = acc1 / num
    acc2 = acc2 / num
    acc3 = acc3 / num

    EPE = np.mean(error)
    return EPE, acc1, acc2, acc3

In [4]:
file_list_test = []
file_list = get_file_list('\\Workspace\\SceneFlow\\datasets\\stereoKITTI_flow_wo_flow_refinement')
for file in file_list:
    file_list_test.append(file)

In [5]:
EPE_all = []
acc1_all = []
acc2_all = []
acc3_all = []

for i, file in enumerate(file_list_test):
    print(i)
    pc_data = np.load(file)
    flow_gt = np.asarray(pc_data['flow_gt'])
    flow_estimate = np.asarray(pc_data['flow_estimate'])
    
    EPE, acc1, acc2, acc3 = scene_flow_EPE_np(flow_estimate, flow_gt)
    EPE_all.append(EPE)
    acc1_all.append(acc1)
    acc2_all.append(acc2)
    acc3_all.append(acc3)
    print(EPE, acc1, acc2, acc3)

0
0.0257004564574704 0.982177734375 0.9954833984375 0.0045166015625
1
0.03666806217861927 0.856689453125 0.935302734375 0.064697265625
2
0.019159494459046245 0.90618896484375 0.99273681640625 0.11322021484375
3
0.014045872010827921 0.9110107421875 0.93267822265625 0.08843994140625
4
0.020173590082592903 0.95733642578125 0.99462890625 0.13201904296875
5
0.02451345023121962 0.81494140625 0.93548583984375 0.16839599609375
6
0.02952700430632123 0.92120361328125 0.99444580078125 0.0657958984375
7
0.040929706949048655 0.8902587890625 0.95147705078125 0.0535888671875
8
0.04693498643910103 0.7474365234375 0.96612548828125 0.14068603515625
9
0.01859446793383467 0.92144775390625 0.9486083984375 0.0513916015625
10
0.026645674737904704 0.8707275390625 0.89703369140625 0.12982177734375
11
0.02016278481252611 0.92559814453125 0.9256591796875 0.1636962890625
12
0.032628790734245704 0.8057861328125 0.92462158203125 0.998291015625
13
0.02284718027999985 0.8797607421875 0.9639892578125 0.03875732421875


0.029399017398025223 0.90216064453125 0.93328857421875 0.1063232421875
127
0.02824687551084647 0.9571533203125 0.9766845703125 0.0234375
128
0.05207377910450296 0.83001708984375 0.83978271484375 0.16021728515625
129
0.020029201991331916 0.96026611328125 0.97601318359375 0.02630615234375
130
0.050129573166428285 0.86517333984375 0.87109375 0.12890625
131
0.009697751109991937 0.998779296875 0.99920654296875 0.00079345703125
132
0.020529191212463806 0.9771728515625 0.99560546875 0.00439453125
133
0.04644792208698241 0.789794921875 0.939697265625 0.060302734375
134
0.0772337702969535 0.82330322265625 0.84295654296875 0.1619873046875
135
0.014695551349858706 0.974853515625 0.974853515625 0.025390625
136
0.05376961767906074 0.663818359375 0.98358154296875 0.01922607421875
137
0.018841296706549576 0.92974853515625 0.982421875 0.03900146484375
138
0.02573785556765899 0.97113037109375 0.9814453125 0.07354736328125
139
0.008484296015773059 0.94757080078125 0.9931640625 0.03009033203125
140
0.020

In [6]:
idx = [2, 3, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
       40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73,
       74, 75, 76, 77, 78, 79, 80, 81, 83, 84, 85, 86, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 105, 106, 107, 108, 109, 110, 111, 112, 113,
       114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 155, 157, 158, 159, 160, 161, 162, 163, 164, 168, 169, 199]
idx = np.array(idx, dtype=int)
EPE_all = np.array(EPE_all)[idx]
acc1_all = np.array(acc1_all)[idx]
acc2_all = np.array(acc2_all)[idx]
acc3_all = np.array(acc3_all)[idx]

In [7]:
print(np.mean(EPE_all), np.mean(acc1_all), np.mean(acc2_all), np.mean(acc3_all))

0.054167486304883744 0.862122441681338 0.9359285596390845 0.23026931117957747
